<a href="https://colab.research.google.com/github/atulpatellnt/My_Test_Repo/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d tmdb/tmdb-movie-metadata

!unzip tmdb-movie-metadata.zip


Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other
Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [3]:
import pandas as pd
import ast

# Load the datasets
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')

# Merge on 'id' , suffixes help to preserve original column names if they are same in both dataframes
movies = movies.merge(credits, left_on='id', right_on='movie_id', suffixes=('_movies', '_credits'))

# Parse 'cast' and get the lead actor
def get_lead_actor(cast_str):
    try:
        cast = ast.literal_eval(cast_str)
        return cast[0]['name'] if cast else ''
    except:
        return ''

# Parse 'genres' and convert to a list of names
def parse_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return [g['name'] for g in genres]
    except:
        return []

# Apply parsing functions
movies['hero'] = movies['cast'].apply(get_lead_actor)
movies['genres'] = movies['genres'].apply(parse_genres)

# Combine title, overview, and hero for the input text
# Use 'title_movies' if 'title' is not found after merge
movies['text'] = movies['title_movies'] + " " + movies['overview'] + " " + movies['hero']

# Select required columns and drop any rows with missing data
df = movies[['text', 'genres']].dropna()
df = df[df['genres'].map(len) > 0]  # Remove rows with empty genre lists

df.head()

,text,genres
0,"Avatar In the 22nd century, a paraplegic Marin...","[Action, Adventure, Fantasy, Science Fiction]"
1,Pirates of the Caribbean: At World's End Capta...,"[Adventure, Fantasy, Action]"
2,Spectre A cryptic message from Bond’s past sen...,"[Action, Adventure, Crime]"
3,The Dark Knight Rises Following the death of D...,"[Action, Crime, Drama, Thriller]"
4,"John Carter John Carter is a war-weary, former...","[Action, Adventure, Science Fiction]"


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
# Import the make_pipeline function
from sklearn.pipeline import make_pipeline

# Convert genres to multi-label binary format
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['genres'])
X = df['text']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline: TF-IDF + OneVsRestClassifier(LogisticRegression)
model = make_pipeline(
    TfidfVectorizer(stop_words='english', max_features=10000),
    OneVsRestClassifier(LogisticRegression(max_iter=1000))
)

# Train the model
model.fit(X_train, y_train)

# Predict genres on test data
y_pred = model.predict(X_test)

# Evaluate the model
print("📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0))

📊 Classification Report:
                 precision    recall  f1-score   support

         Action       0.86      0.20      0.33       242
      Adventure       1.00      0.04      0.07       156
      Animation       0.00      0.00      0.00        39
         Comedy       0.78      0.38      0.51       331
          Crime       1.00      0.03      0.05       160
    Documentary       0.00      0.00      0.00        15
          Drama       0.72      0.69      0.70       445
         Family       0.00      0.00      0.00        97
        Fantasy       0.00      0.00      0.00        94
        Foreign       0.00      0.00      0.00         8
        History       0.00      0.00      0.00        34
         Horror       1.00      0.02      0.04       108
          Music       0.00      0.00      0.00        39
        Mystery       0.00      0.00      0.00        84
        Romance       0.64      0.05      0.09       184
Science Fiction       1.00      0.04      0.09       112
     

In [6]:
# Example custom input
title = "Time Chase"
description = "ssage from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE."
hero=""
# Example custom input
title = "Time Chase"
description = "ssage from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE."
hero=""

# Combine into a single text input
text_input = f"{title} {description} {hero}"

# Predict genre
pred = model.predict([text_input])
predicted_genres = mlb.inverse_transform(pred)

print("Predicted Genres:", predicted_genres[0])
pred = model.predict([text_input])
predicted_genres = mlb.inverse_transform(pred)

print("Predicted Genres:", predicted_genres[0])


Predicted Genres: ('Action', 'Thriller')
Predicted Genres: ('Action', 'Thriller')


In [7]:
import pickle

# Save model and label binarizer
pickle.dump(model, open("genre_model.pkl", "wb"))
pickle.dump(mlb, open("genre_mlb.pkl", "wb"))


In [8]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [13]:


#!pip install -q gradio

import gradio as gr
import pickle

# Load the saved model and label binarizer
try:
    with open("genre_model.pkl", "rb") as f:
        model = pickle.load(f)
    with open("genre_mlb.pkl", "rb") as f:
        mlb = pickle.load(f)
except FileNotFoundError:
    print("Model files not found. Please ensure 'genre_model.pkl' and 'genre_mlb.pkl' are available.")
    # Exit or handle the error appropriately if model files are missing
    exit()


# Define the prediction function for Gradio
def predict_genre(title, description, hero):
    text_input = f"{title} {description} {hero}"
    pred = model.predict([text_input])
    predicted_genres = mlb.inverse_transform(pred)
    return ", ".join(predicted_genres[0]) if predicted_genres[0] else "No specific genre predicted"

# Create the Gradio interface
with gr.Blocks(css="""
    body {
        font-family: 'Arial', sans-serif;
        background: linear-gradient(to right, #6a11cb 0%, #2575fc 100%);
        color: #333;
        display: flex;
        justify-content: center;
        align-items: center;
        min-height: 100vh;
        padding: 20px;
    }
    .gradio-container {
        background-color: #fff;
        padding: 30px;
        border-radius: 15px;
        box-shadow: 0 10px 20px rgba(0, 0, 0, 0.2);
        text-align: center;
        max-width: 700px;
        width: 100%;
        overflow: hidden;
    }
    h1 {
        color: #333;
        margin-bottom: 20px;
        font-size: 2.5em;
        font-weight: bold;
        text-shadow: 1px 1px 2px rgba(0, 0, 0, 0.1);
    }
    .gr-input, .gr-output-text {
        border-radius: 8px !important;
        border: 1px solid #ccc !important;
        padding: 10px !important;
        margin-bottom: 15px !important;
        font-size: 1em !important;
    }
    .gr-button {
        background-color: #4CAF50 !important;
        color: white !important;
        padding: 12px 25px !important;
        border: none !important;
        border-radius: 8px !important;
        cursor: pointer !important;
        font-size: 1.1em !important;
        transition: background-color 0.3s ease !important;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1) !important;
    }
    .gr-button:hover {
        background-color: #45a049 !important;
    }
    .gr-output-text {
        font-size: 1.2em !important;
        color: #007bff !important;
        font-weight: bold !important;
        margin-top: 20px !important;
        background-color: #e9ecef !important;
        padding: 15px !important;
        border-radius: 8px !important;
        min-height: 50px !important; /* Ensure consistent height */
        display: flex !important;
        align-items: center !important;
        justify-content: center !important;
    }
    .animated-title {
        animation: fadeIn 2s ease-in-out infinite alternate;
    }
    @keyframes fadeIn {
        0% { opacity: 0.6; }
        100% { opacity: 1; }
    }
    """) as demo:
    gr.Markdown("<h1 class='animated-title'>🎬 Movie Genre Predictor 🎬</h1>")
    gr.Markdown("Enter details about a movie and let the model predict its genre(s).")

    with gr.Row():
        title_input = gr.Textbox(label="Movie Title", placeholder="e.g., The Matrix")
        hero_input = gr.Textbox(label="Lead Actor (Optional)", placeholder="e.g., Keanu Reeves")

    description_input = gr.Textbox(label="Movie Overview/Description", lines=4, placeholder="e.g., A computer hacker learns about the true nature of his reality...")

    predict_button = gr.Button("Predict Genres")

    output_text = gr.Textbox(label="Predicted Genre(s)", interactive=False)

    predict_button.click(
        fn=predict_genre,
        inputs=[title_input, description_input, hero_input],
        outputs=output_text
    )

    gr.Markdown("Built by L&TEdutech")

# Launch the interface
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b73099fac203a8fe68.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Please Click Here**  https://b73099fac203a8fe68.gradio.live